# PredNet

Train PredNet on KITTI sequences. (Geiger et al. 2013, http://www.cvlibs.net/datasets/kitti/)

* Python 3.5.2
* tensorflow-gpu 0.12.1
* Keras 1.0.8

In [ ]:
from __future__ import print_function

import os
import numpy as np
np.random.seed(123)
from six.moves import cPickle

from keras import backend as K
from keras.models import Model
from keras.layers import Input, Dense, Flatten
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam

from prednet import PredNet
from data_utils import SequenceGenerator
from kitti_settings import *


Using TensorFlow backend.


In [ ]:
## Model files
save_model = True
weights_file = os.path.join(WEIGHTS_DIR, 'toke_weights.hdf5')
json_file = os.path.join(WEIGHTS_DIR, 'toke_model.json')

In [ ]:
## Data files
train_file = os.path.join(DATA_DIR, 'X_train.hkl')
train_sources = os.path.join(DATA_DIR, 'sources_train.hkl')
val_file = os.path.join(DATA_DIR, 'X_val.hkl')
val_sources = os.path.join(DATA_DIR, 'sources_val.hkl')

In [ ]:
## Training parameters
nb_epoch = 150
batch_size = 4
samples_per_epoch = 500
N_seq_val = 100  # number of sequences to use for validation


In [ ]:
## Model parameters
nt = 10 # Number of timesteps per sequence
n_channels, im_height, im_width = (3, 128, 160) #input shape
if K.image_dim_ordering() == 'th':
    input_shape = (n_channels, im_height, im_width)
else:
    input_shape = (im_height, im_width, n_channels)
stack_sizes = (n_channels, 48, 96, 192) #feature map sizes
R_stack_sizes = stack_sizes

A_filt_sizes = (3, 3, 3)
Ahat_filt_sizes = (3, 3, 3, 3)
R_filt_sizes = (3, 3, 3, 3)
layer_loss_weights = np.array([1., 0.1, 0.1, 0.1])
layer_loss_weights = np.expand_dims(layer_loss_weights, 1)
time_loss_weights = 1./ (nt - 1) * np.ones((nt,1))
time_loss_weights[0] = 0 # Don't use loss on initial timestep

In [ ]:
## Creating the model
prednet = PredNet(stack_sizes, # Number of channels in Conv layers (A, Ahat)
                  R_stack_sizes, # Number of channels in Conv LSTM layers (R)
                  A_filt_sizes, # Filter size
                  Ahat_filt_sizes, 
                  R_filt_sizes,
                  output_mode='error', 
                  return_sequences=True)

inputs = Input(shape=(nt,) + input_shape)
errors = prednet(inputs)  # errors will be (batch_size, nt, nb_layers)
# errors_by_time = TimeDistributed(
#     Dense(
#         1,
#         weights=[layer_loss_weights, np.zeros(1)], 
#         trainable=False), 
#     trainable=False)(errors)  # calculate weighted error by layer

# errors_by_time = Flatten()(errors_by_time)  # will be (batch_size, nt)
# final_errors = Dense(1,
#     weights=[time_loss_weights, np.zeros(1)],
#     trainable=False)(errors_by_time)  # weight errors by time
# model = Model(input=inputs, output=final_errors)
# model.compile(loss='mean_absolute_error', optimizer='adam')
